# Copyright and disclaimer 

This work is licensed under the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License. To view a copy of this license, visit [this page](http://creativecommons.org/licenses/by-nc-sa/4.0/) or send a letter to Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

The license gives you permission to copy, distribute and modify the code as you see fit provided that your work is intended for non-commercial purposes, you give me and the previous author credit, and you retain the license in all your releases derived from this work.

The code below consists mostly of an implementation of original code developed by and kindly provided by Igor Douven inside a Jupyter notebook environment. I modified the code and wrapped it in terms of functions to facilitate my usage. I make no claims to ease of comprehension or reproductibility.

# Introduction 

The version of the Hegselmann-Krause model implemented by Douven is given by

$$
x_{i}(u+1)=\alpha \frac{1}{\left|X_{i}(u)\right|} \sum_{j \in X_{i}(u)} x_{j}(u)+(1-\alpha)(\tau+\operatorname{rnd}(\zeta))
$$

whereby

$$
X_{i}(u) :=\left\{j :\left|x_{i}(u)-x_{j}(u)\right| \leq \varepsilon\right\}
$$

In what follows, all the code used is copyright of Igor Douven unless stated otherwise.

## Hegselmann-Krause Updater 

In [1]:
# import necessary packages
using Distributions, Gadfly, Compose, DataFrames, StatsBase

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/sebastian/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


In [2]:
# define the updater. see the paper for explanations
#function hk_upd_n(n_agents::Int64, n_steps::Int64, ϵ::Float64, α::Float64, τ::Float64, ζ::Float64, averaging)
function hk_upd_n(n_agents, n_steps, ϵ, α, τ, ζ, averaging)
    hk_ar = Array{Float64,2}(undef, n_agents, n_steps + 1)
    hk_ar[:, 1] = rand(n_agents)
    for j in 1:n_steps, i in 1:n_agents
        fa = isapprox.(hk_ar[:, j], hk_ar[:, j][i], atol=ϵ)
        hk_ar[i, j + 1] = α*averaging(hk_ar[:, j][fa]) + (1 - α)*max(0, min(1, τ + rand(Uniform(-ζ, ζ))))
    end
    return hk_ar
end

# OPEN QUESTION: specifying the data types gives me an error. Why?
# Restarting the kernel... misteriosos son los caminos

hk_upd_n (generic function with 1 method)

In [5]:
# declare the parameter values as used in the 2010 paper
ϵ = 0.1; α = 0.9; τ = 0.75; ζ = 0.2;

In [24]:
# run a trial simulation
res = hk_upd_n(25, 50, ϵ, α, τ, ζ, mean)

25×51 Array{Float64,2}:
 0.704387    0.687666  0.713068  0.68371   …  0.742661  0.746902  0.735675
 0.920065    0.886702  0.850709  0.84337      0.756316  0.726891  0.739545
 0.9633      0.904586  0.868353  0.852855     0.737663  0.737687  0.760751
 0.153066    0.203986  0.240051  0.282689     0.739968  0.746615  0.764251
 0.0939042   0.148917  0.236664  0.286856     0.736811  0.758707  0.740175
 0.201284    0.214741  0.210298  0.275777  …  0.738897  0.75897   0.744536
 0.724583    0.731308  0.702341  0.689783     0.726275  0.756111  0.730175
 0.927988    0.912557  0.877352  0.83106      0.744859  0.7545    0.764038
 0.626182    0.639604  0.699934  0.706371     0.738782  0.73856   0.75736 
 0.107393    0.164756  0.20994   0.265522     0.737083  0.7514    0.752199
 0.756382    0.774432  0.750599  0.726213  …  0.759584  0.722646  0.74115 
 0.0482192   0.137823  0.225566  0.289323     0.72866   0.757042  0.750317
 0.127967    0.16932   0.246888  0.291837     0.739335  0.730725  0.743428
 

In the above array, each row corresponds to an agent whereas each column corroesponds to a time step.

## Data Frames 

The Gadfly package is designed to work best with dataframes. Hence, we're better off converting res into a dataframe object.

In [26]:
df = DataFrame(res') # transpose
names!(df, [Symbol("Agent $i") for i in 1:size(res,1)])
df = stack(df)
df[:steps] = repeat(1:size(res, 2), outer=size(res, 1))
df

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[26]:4
└ @ Core In[26]:4


,variable,value,steps
,Symbol,Float64,Int32
1,Agent 1,0.0332152,1
2,Agent 1,0.0968633,2
3,Agent 1,0.190823,3
4,Agent 1,0.248204,4
5,Agent 1,0.279933,5
6,Agent 1,0.334502,6
7,Agent 1,0.356572,7
8,Agent 1,0.423435,8
9,Agent 1,0.46118,9


## A Plot 

In [4]:
# use this to resize all plots
set_default_plot_size(20cm ,15cm)

In [ ]:
res_plot = plot(df, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(res, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = $τ, ζ = $ζ, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"black"),
    Guide.annotation(compose(context(), Compose.text(size(res, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        # toggle colors appropriate to the theme
        #major_label_color=colorant"grey",
        #minor_label_color=colorant"grey",
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt),
)

Alright! Note that the results are robust for these parameter values under changes in number of agents. After $t \approx 35$, all agents converge on the true value. Let's run a few more tests for the fun of it, shall we?

### Noiseless 

In [ ]:
# running with no noise
#function hk_upd_noiseless(n_agents::Int64, n_steps::Int64, ϵ::Float64, α::Float64, τ::Float64, ζ::Float64, averaging)
function hk_upd_noiseless(n_agents, n_steps, ϵ, α, τ, ζ, averaging)
    hk_ar = Array{Float64,2}(undef, n_agents, n_steps + 1)
    hk_ar[:, 1] = rand(n_agents)
    for j in 1:n_steps, i in 1:n_agents
        fa = isapprox.(hk_ar[:, j], hk_ar[:, j][i], atol=ϵ)
        hk_ar[i, j + 1] = α*averaging(hk_ar[:, j][fa]) + (1 - α)*τ
    end
    return hk_ar
end
test1 = hk_upd_noiseless(25, 50, ϵ, α, τ, 0.0, mean)
df1 = DataFrame(test1') # transpose
names!(df1, [Symbol("Agent $i") for i in 1:size(test1,1)])
df1 = stack(df1)
df1[:steps] = repeat(1:size(test1, 2), outer=size(test1, 1))
test1_plot = plot(df1, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test1, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = $τ, ζ = 0, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(res, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

### Large Noise Short-Term

In [ ]:
test2a = hk_upd_n(25, 50, ϵ, α, τ, 0.75, mean)
df2a = DataFrame(test2a') # transpose
names!(df2a, [Symbol("Agent $i") for i in 1:size(test2a,1)])
df2a = stack(df2a)
df2a[:steps] = repeat(1:size(test2a, 2), outer=size(test2a, 1))
size(df2a)

In [ ]:
test2a_plot = plot(df2a, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test2a, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = $τ, ζ = 0.75, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(test2a, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

### Large Noise Long-Term 

In [ ]:
test2 = hk_upd_n(25, 500, ϵ, α, τ, 0.75, mean)
df2 = DataFrame(test2') # transpose
names!(df2, [Symbol("Agent $i") for i in 1:size(test2,1)])
df2 = stack(df2)
df2[:steps] = repeat(1:size(test2, 2), outer=size(test2, 1))

In [ ]:
test2_plot = plot(df2, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test2, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = $τ, ζ = 0.75, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(test2, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

The above result suggests that given a large enough value of $\zeta$, the agents fail to converge at all to the true value of $\tau$.

### Large Tolerance

In [ ]:
test3 = hk_upd_n(25, 50, 0.9, α, τ, ζ, mean)
df3 = DataFrame(test3') # transpose
names!(df3, [Symbol("Agent $i") for i in 1:size(test3,1)])
df3 = stack(df3)
df3[:steps] = repeat(1:size(test3, 2), outer=size(test3, 1))
test3_plot = plot(df3, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test3, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = 0.9, α = $α, τ = $τ, ζ = $ζ, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(test3, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

The above suggests that increasing the tolerance interval $\epsilon$ among agents leads to a much rapid convergence.

### Other True Parameter Value

In [ ]:
test4 = hk_upd_n(25, 50, ϵ, α, 0.2, ζ, mean)
df4 = DataFrame(test4') # transpose
names!(df4, [Symbol("Agent $i") for i in 1:size(test4,1)])
df4 = stack(df4)
df4[:steps] = repeat(1:size(test4, 2), outer=size(test4, 1))
test4_plot = plot(df4, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test4, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = 0.2, ζ = $ζ, mean"),
    yintercept=[0.2], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(test4, 2) + 1, 0.2 + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

This one is obvious, but still done for completeness.

### Small Weight to Peers

In [ ]:
test5 = hk_upd_n(25, 50, ϵ, 0.1, τ, ζ, mean)
df5 = DataFrame(test5') # transpose
names!(df5, [Symbol("Agent $i") for i in 1:size(test5,1)])
df5 = stack(df5)
df5[:steps] = repeat(1:size(test5, 2), outer=size(test5, 1))
test5_plot = plot(df5, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(test5, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = 0.1, τ = $τ, ζ = $ζ, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"grey"),
    Guide.annotation(compose(context(), Compose.text(size(test5, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt))

If the weight $\alpha$ assigned to peers is too small, the agents will approach the neighborhood of the true value, but will never get closer past a threshold the size of $\zeta$.

## Generalizing Simulations

In [3]:
# this function will run the HK model 100 times
#function hk_sim_n(n_agents::Int64, n_steps::Int64, ϵ::Float64, α::Float64, τ::Float64, ζ::Float64, averaging; n_sim::Int64=100)
function hk_sim_n(n_agents, n_steps, ϵ, α, τ, ζ, averaging, n_sim)
    sim_res = Array{Float64,3}(undef, n_agents, n_steps + 1, n_sim)
    for i in 1:n_sim
        sim_res[:, :, i] = hk_upd_n(n_agents, n_steps, ϵ, α, τ, ζ, averaging)
    end
    return sim_res
end

hk_sim_n (generic function with 1 method)

In [ ]:
sims = hk_sim_n(25, 50, ϵ, α, τ, ζ, mean, 100)

## Descriptive Statistics 

In [ ]:
# how far removed from the truth are the agents at each step (squared error)?
dts = map(x->(x-τ)^2, sims)

In [ ]:
#what is the sum of the squared errors for the various agents in the various simulations?
sum(dts, dims=2)

In [ ]:
#on average, how far removed are the agents from the truth, at each step in each simulation?
mean(dts, dims=1)

## Table 2 Comparison

In [45]:
sim_res = hk_sim_n(25, 50, 0.1, 0.9, τ, ζ, mean, 100)
# find the square error from the truth of each agent at each step, in each simulation
square_error = map(x->2*(x-τ)^2, sim_res)
# find the average distance of the agents from the truth at each step, in each simulation
step_average = mean(square_error, dims=1)

1×51×100 Array{Float64,3}:
[:, :, 1] =
 0.338573  0.269319  0.217164  0.172285  …  0.000204003  0.000254325

[:, :, 2] =
 0.312764  0.256168  0.204701  0.164495  …  0.000393963  0.000327243

[:, :, 3] =
 0.249342  0.2052  0.166056  0.13381  0.110359  …  0.000322345  0.000338708

...

[:, :, 98] =
 0.373438  0.30045  0.242963  0.189475  0.157576  …  0.000252691  0.000175549

[:, :, 99] =
 0.265107  0.213355  0.167372  0.130283  …  0.000281534  0.000217797

[:, :, 100] =
 0.27233  0.217669  0.16998  0.138159  0.114076  …  0.000364499  0.000369818

In [46]:
t = 50
# find the average distance from the truth at the fifth step of all simulations
fifthstep_average = mean(step_average[1,5,:])

0.12358522616922643

In [47]:
# find the average distance from the truth at the tenth step of all simulations
tenthstep_average = mean(step_average[1,10,:])

0.041500910257698774

In [48]:
# find the average distance from the truth at the tenth step of all simulations
twentyfifthstep_average = mean(step_average[1,25,:])

0.001309327340820676

In [49]:
# find the average distance from the truth at the last step of all simulations
fiftieth_average = mean(step_average[1,t+1,:])

0.00031237849256472467

In [63]:
# no difference-splitting
sim_res = hk_sim_n(25, 50, 0, 0.5, τ, ζ, mean, 100)
square_error = map(x->2*(x-τ)^2, sim_res)
# find the average distance of the agents from the truth at each step, in each simulation
step_average = mean(square_error, dims=1)

1×51×100 Array{Float64,3}:
[:, :, 1] =
 0.326621  0.094088  0.0362845  …  0.0086383  0.00587286  0.00696172

[:, :, 2] =
 0.246988  0.0533916  0.019233  …  0.00879059  0.00950248  0.00969632

[:, :, 3] =
 0.214866  0.0614204  0.0162417  …  0.00983783  0.0087946  0.00817835

...

[:, :, 98] =
 0.218168  0.0424686  0.0164979  …  0.00489704  0.0113827  0.0127369

[:, :, 99] =
 0.377968  0.097092  0.0381848  …  0.0110523  0.00846974  0.0108067

[:, :, 100] =
 0.179066  0.0554133  0.0268531  …  0.0113538  0.00682988  0.00418005

In [64]:
t = 50
# find the average distance from the truth at the fifth step of all simulations
fifthstep_average = mean(step_average[1,5,:])

0.010396946895966111

In [65]:
# find the average distance from the truth at the tenth step of all simulations
tenthstep_average = mean(step_average[1,10,:])

0.008906998358351044

In [66]:
# find the average distance from the truth at the tenth step of all simulations
twentyfifthstep_average = mean(step_average[1,25,:])

0.00903139789148511

In [67]:
# find the average distance from the truth at the last step of all simulations
fiftieth_average = mean(step_average[1,t+1,:])

0.00866499307864603

## Helper Functions 

These functions will offer generalizations of Douven's Code to facilitate data processing and plotting

In [18]:
# process simulation result to make it plotable in Gadfly
function stack_dataset(hk_array)
    dframe = DataFrame(hk_array')
    names!(dframe, [Symbol("Agent $i") for i in 1:size(hk_array,1)])
    dframe = stack(dframe)
    dframe[:steps] = repeat(1:size(hk_array, 2), outer=size(hk_array, 1))
    return dframe
end

stack_dataset (generic function with 1 method)

In [30]:
# test the helper function
# note: there is some deprecated whatever, but I don't care
test_dframe = stack_dataset(res)
test_dframe

,variable,value,steps
,Symbol,Float64,Int32
1,Agent 1,0.0332152,1
2,Agent 1,0.0968633,2
3,Agent 1,0.190823,3
4,Agent 1,0.248204,4
5,Agent 1,0.279933,5
6,Agent 1,0.334502,6
7,Agent 1,0.356572,7
8,Agent 1,0.423435,8
9,Agent 1,0.46118,9


In [20]:
function plot_dataset(hk_array)
    dframe = stack_dataset(hk_array)
    plot_result = plot(dframe, x=:steps, y=:value, color=:variable, Geom.point, Geom.line,
    Coord.cartesian(xmax=size(hk_array, 2) + 1),
    Guide.xlabel("Time Step"),
    Guide.ylabel("Believed True Value"),
    Guide.title("Trial Run with Parameters: ϵ = $ϵ, α = $α, τ = $τ, ζ = $ζ, mean"),
    yintercept=[τ], Geom.hline(style=:dot, color=colorant"black"),
    Guide.annotation(compose(context(), Compose.text(size(hk_array, 2) + 1, τ + 0.01, "τ"), font("DejaVu Mono"), fontsize(10pt))),
    Theme(key_position=:none, point_size=1.25pt,
        background_color=colorant"white",
        major_label_font="DejaVu Mono",
        minor_label_font="DejaVu Mono",
        major_label_font_size=12pt,
        minor_label_font_size=8pt),)
    return plot_result
end

plot_dataset (generic function with 1 method)

In [ ]:
plot_dataset(res)

Wonderful, this does all I want it to do! :)